In [25]:
# control the browser 
from selenium import webdriver
# simulate keyboard presses
from selenium.webdriver.common.keys import Keys
# locate elements using various methods include: name, class name, tag name, CSS selector, XPath
from selenium.webdriver.common.by import By
# for implementing wait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

- For this form, labels are unique and appears before it's corresponding input field. 
- Order-independent. Using label-based XPath with `following::` axis to find the correct input regardless of the form's order. Even if the questions are rearranged, it will still find the correct field as long as the label text remains the same. 
- Works for both single input fields and multi-line textareas at one go.

In [26]:
labels = ["Name","Email","Phone number","Address","Comments"] 

In [27]:
classList = [
  ['JDE student','JDE@gmail.com','123456789','2474 Hong Kong SAR','NA'],
  ['JDE Instructor','JDE2@gmail.com','987654321','2143 Singapore','NA']
]

In [28]:
# launch browser instance
driver = webdriver.Chrome() 
#navigate to the site
URL = "https://docs.google.com/forms/d/e/1FAIpQLSeYmAdf649WvdEBxbaVxt2kWlxifYogaVk3kS4IjUW0E5zWsQ/viewform"

In [29]:
TIMEOUT = 5

In [30]:
# find the input field/text area on the form that matches the given label, return the xpath so we can interact with it
def get_field_by_label(driver, label_text):
    
    # find <div role="heading" that contains the label
    # get either <input> or <textarea> that comes after the label
    xpath = (
        "("        
            f"//div[@role='heading'][.//span[text()='{label_text}']]/" 
            "following::input[@jsname='YPqjbf']"    
            "|"
            f"//div[@role='heading'][.//span[text()='{label_text}']]/"
            "following::textarea[@jsname='YPqjbf']"
        ")"  
    )

    return driver.find_element(By.XPATH, xpath)

In [31]:
def main():
    
    for data in classList:
        # navigate to specified URL in the chrome browser       
        driver.get(URL)

        # add delay
        # time.sleep(5)
        # locating form element, assuming its child elements (textinputs, buttons) are likely ready too
        WebDriverWait(driver,TIMEOUT).until(
            EC.presence_of_element_located((By.ID,"mG61Hd"))
        )
          
        for label,value in zip(labels,data):
            print(f"{label} -> {value}")
                   
            # pass selenium browser instance, current label e.g. Name to build xpath
            field = get_field_by_label(driver,label)
            
            field.clear() #clear any existing text
            field.send_keys(value) #type into the field

        # Finds the <span> element on the page that contains the text “Submit” and click it.
        driver.find_element(By.XPATH, "//span[text()='Submit']").click()

        # Pause and wait until the form has fully submitted and redirected to the confirmation page before moving on to next record
        # Since Google Forms always redirects to a URL containing "formResponse". (https://docs.google.com/forms/.........../formResponse)
        WebDriverWait(driver, TIMEOUT).until(
            EC.url_contains("formResponse")
        )

    # close browser window and end webdriver session
    driver.quit()    

In [32]:
if __name__=="__main__":
    main()

Name -> JDE student
Email -> JDE@gmail.com
Phone number -> 123456789
Address -> 2474 Hong Kong SAR
Comments -> NA
Name -> JDE Instructor
Email -> JDE2@gmail.com
Phone number -> 987654321
Address -> 2143 Singapore
Comments -> NA
